In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
import os
import math
import numpy as np
from glob import glob

In [2]:
def read_folder(folder_path) : # jpg 파일 목록 반환
    file_list_all = os.listdir(folder_path)
    file_list = [file for file in file_list_all if file.endswith('jpg')]
    return file_list

# 각 sequece에서 원하는 만큼의 이미지를 추출
def data_maker(all_images, n) :

    selected_images = []
    if n == 7 :
        selected_images = all_images

    else :
        # 7장의 이미지 중 가운데 n장만 선택
        for i in range(0, len(all_images), 7):
            
            start = i + int((7 - n) / 2)
            end = i + int((7 + n) / 2)
           
            selected_images.extend(all_images[start:end])

    images = [image.load_img(p, target_size=(224, 224)) for p in selected_images]
    vector = np.asarray([image.img_to_array(img) for img in images])

    return vector

In [3]:
# 이미지 데이터 경로

test_CA_Y_folder = read_folder("./test/image/CA_Y")
test_CA_N_folder = read_folder("./test/image/CA_N")
# test_CA_Y_folder = read_folder("./temp_test/CA_Y")
# test_CA_N_folder = read_folder("./temp_test/CA_N")

train_CA_Y_folder = read_folder("./train/image/CA_Y")
train_CA_N_folder = read_folder("./train/image/CA_N")
# train_CA_Y_folder = read_folder("./temp_train/CA_Y")
# train_CA_N_folder = read_folder("./temp_train/CA_N")

# 학대 => 1, 학대 x => 0으로  라벨링데이터 생성
# labels = ([1] * int(len(CA_Y_folder)/7)) + ([0] * int(len(CA_N_folder)/7))
# print(len(labels))
test_label = ([1] * int(len(test_CA_Y_folder)/7)) + ([0] * int(len(test_CA_N_folder)/7))
train_label = ([1] * int(len(train_CA_Y_folder)/7)) + ([0] * int(len(train_CA_N_folder)/7))
print(len(test_label))
print(len(train_label))

# 전체 이미지 데이터 불러오기
# all_images = glob('./CA_Y/*jpg') + glob('./CA_N/*jpg')


# test_images = glob('./temp_test/CA_Y/*jpg') + glob('./temp_test/CA_N/*jpg')
# train_images = glob('./temp_train/CA_Y/*jpg') + glob('./temp_train/CA_N/*jpg')
test_images = glob('./test/image/CA_Y/*jpg') + glob('./test/image/CA_N/*jpg')
train_images = glob('./train/image/CA_Y/*jpg') + glob('./train/image/CA_N/*jpg')

165
1256


In [4]:
base_model = VGG16(weights='imagenet', include_top=True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

In [5]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc1 (Dense)                          │ (None, 4096)                │     102,764,544 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 134,260,544 (512.16 MB)

 Trainable params: 134,260,544 (512.16 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
image_per_sequence = 1
# vector = data_maker(all_images, image_per_sequence)
# vector.shape

test_vector = data_maker(test_images, image_per_sequence)
train_vector = data_maker(train_images, image_per_sequence)

In [7]:
# 개별 이미지에서 벡터를 추출
# vectors = model.predict(preprocess_input(vector)) 

test_vectors = model.predict(preprocess_input(test_vector))
train_vectors = model.predict(preprocess_input(train_vector))

6/6 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 147s 4s/step


In [8]:
# vectors.shape

print(test_vectors.shape)
print(train_vectors.shape)

(165, 4096)
(1256, 4096)


In [9]:
# concatenated_vectors = np.array([])
test_concatenated_vectors = np.array([])
train_concatenated_vectors = np.array([])

# (이미지 수, 4096) 형태의 배열을 n개씩 묶어서 (이미지수 / n, 4096 * n) 벡터를 만드는 함수
def concatenate_vectors(vectors, n):
    concatenated_vectors = []
    for i in range(0, vectors.shape[0], n):
        concatenated = vectors[i:i+n].reshape(1, -1)
        concatenated_vectors.append(concatenated)
    return np.vstack(concatenated_vectors)

# 함수 호출
# concatenated_vectors = concatenate_vectors(vectors, image_per_sequence)
test_concatenated_vectors = concatenate_vectors(test_vectors, image_per_sequence)
train_concatenated_vectors = concatenate_vectors(train_vectors, image_per_sequence)
    
# 결과 출력
# print(concatenated_vectors.shape)  
print(f"test_concatenated_vectors : {test_concatenated_vectors.shape}")  
print(f"train_concatenated_vectors : {train_concatenated_vectors.shape}")  

test_concatenated_vectors : (165, 4096)
train_concatenated_vectors : (1256, 4096)


In [10]:
# from sklearn.model_selection import train_test_split  
# X_train, X_test, y_train, y_test = train_test_split(concatenated_vectors, labels, test_size=0.2)

In [11]:
import numpy as np

test_label = np.array(test_label)
train_label = np.array(train_label)

# 랜덤한 인덱스를 생성
test_indices = np.random.permutation(len(test_concatenated_vectors))
train_indices = np.random.permutation(len(train_concatenated_vectors))

# 인덱스를 사용하여 배열을 섞음
test_concatenated_shuffled_vectors = test_concatenated_vectors[test_indices]
test_shuffled_label = test_label[test_indices]

train_concatenated_shuffled_vectors = train_concatenated_vectors[train_indices]
train_shuffled_label = train_label[train_indices]

print(train_shuffled_label)

[1 1 1 ... 1 1 0]


In [12]:
from sklearn.linear_model import LogisticRegression
lr_p1 = LogisticRegression(C=0.1, penalty='l1', solver='saga', max_iter=10000) 
lr_p1.fit(train_concatenated_shuffled_vectors, train_shuffled_label)
print(lr_p1.score(test_concatenated_shuffled_vectors, test_shuffled_label))

0.9393939393939394


In [13]:
import joblib

# 모형 저장하기
model_name = f"Model2_Seq{image_per_sequence}_VGG16.joblib"
joblib.dump(lr_p1, model_name)

['Model2_Seq1_VGG16.joblib']

In [14]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import joblib

image_per_sequence = 3

test_vector = data_maker(test_images, image_per_sequence)
train_vector = data_maker(train_images, image_per_sequence)
print(f"test_vector : {test_vector.shape}")  
print(f"train_vector : {train_vector.shape}")  
print("")

# 개별 이미지에서 벡터를 추출
test_vectors = model.predict(preprocess_input(test_vector))
train_vectors = model.predict(preprocess_input(train_vector))
print(f"test_vectors : {test_vector.shape}")  
print(f"train_vectors : {train_vector.shape}")
print("")

# concatenation
test_concatenated_vectors = np.array([])
train_concatenated_vectors = np.array([])

# (이미지 수, 4096) 형태의 배열을 n개씩 묶어서 (이미지수 / n, 4096 * n) 벡터를 만드는 함수
def concatenate_vectors(vectors, n):
    concatenated_vectors = []
    for i in range(0, vectors.shape[0], n):
        concatenated = vectors[i:i+n].reshape(1, -1)
        concatenated_vectors.append(concatenated)
    return np.vstack(concatenated_vectors)

# 함수 호출
test_concatenated_vectors = concatenate_vectors(test_vectors, image_per_sequence)
train_concatenated_vectors = concatenate_vectors(train_vectors, image_per_sequence)
print(f"test_concatenated_vectors : {test_concatenated_vectors.shape}")  
print(f"train_concatenated_vectors : {train_concatenated_vectors.shape}")
print("")

# 데이터 shuffling
test_label = np.array(test_label)
train_label = np.array(train_label)

test_indices = np.random.permutation(len(test_concatenated_vectors))
train_indices = np.random.permutation(len(train_concatenated_vectors))

test_concatenated_shuffled_vectors = test_concatenated_vectors[test_indices]
test_shuffled_label = test_label[test_indices]

train_concatenated_shuffled_vectors = train_concatenated_vectors[train_indices]
train_shuffled_label = train_label[train_indices]


# 로지스틱 회귀모형 학습 
lr_p1 = LogisticRegression(C=0.1, penalty='l1', solver='saga', max_iter=10000) 
lr_p1.fit(train_concatenated_shuffled_vectors, train_shuffled_label)
print(lr_p1.score(test_concatenated_shuffled_vectors, test_shuffled_label))

# 모형 저장하기
model_name = f"Model2_Seq{image_per_sequence}_VGG16.joblib"
joblib.dump(lr_p1, model_name)

test_vector : (495, 224, 224, 3)
train_vector : (3768, 224, 224, 3)

16/16 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step
118/118 ━━━━━━━━━━━━━━━━━━━━ 405s 3s/step
test_vectors : (495, 224, 224, 3)
train_vectors : (3768, 224, 224, 3)

test_concatenated_vectors : (165, 12288)
train_concatenated_vectors : (1256, 12288)

0.9393939393939394


['Model2_Seq3_VGG16.joblib']

In [15]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import joblib

image_per_sequence = 5

test_vector = data_maker(test_images, image_per_sequence)
train_vector = data_maker(train_images, image_per_sequence)
print(f"test_vector : {test_vector.shape}")  
print(f"train_vector : {train_vector.shape}")  
print("")

# 개별 이미지에서 벡터를 추출
test_vectors = model.predict(preprocess_input(test_vector))
train_vectors = model.predict(preprocess_input(train_vector))
print(f"test_vectors : {test_vector.shape}")  
print(f"train_vectors : {train_vector.shape}")
print("")

# concatenation
test_concatenated_vectors = np.array([])
train_concatenated_vectors = np.array([])

# (이미지 수, 4096) 형태의 배열을 n개씩 묶어서 (이미지수 / n, 4096 * n) 벡터를 만드는 함수
def concatenate_vectors(vectors, n):
    concatenated_vectors = []
    for i in range(0, vectors.shape[0], n):
        concatenated = vectors[i:i+n].reshape(1, -1)
        concatenated_vectors.append(concatenated)
    return np.vstack(concatenated_vectors)

# 함수 호출
test_concatenated_vectors = concatenate_vectors(test_vectors, image_per_sequence)
train_concatenated_vectors = concatenate_vectors(train_vectors, image_per_sequence)
print(f"test_concatenated_vectors : {test_concatenated_vectors.shape}")  
print(f"train_concatenated_vectors : {train_concatenated_vectors.shape}")
print("")


# 데이터 shuffling
test_label = np.array(test_label)
train_label = np.array(train_label)

test_indices = np.random.permutation(len(test_concatenated_vectors))
train_indices = np.random.permutation(len(train_concatenated_vectors))

test_concatenated_shuffled_vectors = test_concatenated_vectors[test_indices]
test_shuffled_label = test_label[test_indices]

train_concatenated_shuffled_vectors = train_concatenated_vectors[train_indices]
train_shuffled_label = train_label[train_indices]


# 로지스틱 회귀모형 학습 
lr_p1 = LogisticRegression(C=0.1, penalty='l1', solver='saga', max_iter=10000) 
lr_p1.fit(train_concatenated_shuffled_vectors, train_shuffled_label)
print(lr_p1.score(test_concatenated_shuffled_vectors, test_shuffled_label))

# 모형 저장하기
model_name = f"Model2_Seq{image_per_sequence}_VGG16.joblib"
joblib.dump(lr_p1, model_name)

test_vector : (825, 224, 224, 3)
train_vector : (6280, 224, 224, 3)

26/26 ━━━━━━━━━━━━━━━━━━━━ 97s 4s/step
197/197 ━━━━━━━━━━━━━━━━━━━━ 685s 3s/step
test_vectors : (825, 224, 224, 3)
train_vectors : (6280, 224, 224, 3)

test_concatenated_vectors : (165, 20480)
train_concatenated_vectors : (1256, 20480)

0.9333333333333333


['Model2_Seq5_VGG16.joblib']

In [16]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import joblib

image_per_sequence = 7

test_vector = data_maker(test_images, image_per_sequence)
train_vector = data_maker(train_images, image_per_sequence)
print(f"test_vector : {test_vector.shape}")  
print(f"train_vector : {train_vector.shape}")  
print("")

# 개별 이미지에서 벡터를 추출
test_vectors = model.predict(preprocess_input(test_vector))
train_vectors = model.predict(preprocess_input(train_vector))
print(f"test_vectors : {test_vector.shape}")  
print(f"train_vectors : {train_vector.shape}")
print("")

# concatenation
test_concatenated_vectors = np.array([])
train_concatenated_vectors = np.array([])

# (이미지 수, 4096) 형태의 배열을 n개씩 묶어서 (이미지수 / n, 4096 * n) 벡터를 만드는 함수
def concatenate_vectors(vectors, n):
    concatenated_vectors = []
    for i in range(0, vectors.shape[0], n):
        concatenated = vectors[i:i+n].reshape(1, -1)
        concatenated_vectors.append(concatenated)
    return np.vstack(concatenated_vectors)

# 함수 호출
test_concatenated_vectors = concatenate_vectors(test_vectors, image_per_sequence)
train_concatenated_vectors = concatenate_vectors(train_vectors, image_per_sequence)
print(f"test_concatenated_vectors : {test_concatenated_vectors.shape}")  
print(f"train_concatenated_vectors : {train_concatenated_vectors.shape}")
print("")


# 데이터 shuffling
test_label = np.array(test_label)
train_label = np.array(train_label)

test_indices = np.random.permutation(len(test_concatenated_vectors))
train_indices = np.random.permutation(len(train_concatenated_vectors))

test_concatenated_shuffled_vectors = test_concatenated_vectors[test_indices]
test_shuffled_label = test_label[test_indices]

train_concatenated_shuffled_vectors = train_concatenated_vectors[train_indices]
train_shuffled_label = train_label[train_indices]


# 로지스틱 회귀모형 학습 
lr_p1 = LogisticRegression(C=0.1, penalty='l1', solver='saga', max_iter=10000) 
lr_p1.fit(train_concatenated_shuffled_vectors, train_shuffled_label)
print(lr_p1.score(test_concatenated_shuffled_vectors, test_shuffled_label))

# 모형 저장하기
model_name = f"Model2_Seq{image_per_sequence}_VGG16.joblib"
joblib.dump(lr_p1, model_name)

test_vector : (1155, 224, 224, 3)
train_vector : (8792, 224, 224, 3)

37/37 ━━━━━━━━━━━━━━━━━━━━ 117s 3s/step
275/275 ━━━━━━━━━━━━━━━━━━━━ 532s 2s/step
test_vectors : (1155, 224, 224, 3)
train_vectors : (8792, 224, 224, 3)

test_concatenated_vectors : (165, 28672)
train_concatenated_vectors : (1256, 28672)

0.9333333333333333


['Model2_Seq7_VGG16.joblib']

In [17]:
# 모형 사용하기
# loaded_model = joblib.load('Model2.joblib')

# 불러온 모델로 예측
# predictions = loaded_model.predict(X_test)
# print(predictions)

In [18]:

# import numpy as np
#import matplotlib.pyplot as plt
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import log_loss
#from sklearn.model_selection import train_test_split

# 데이터 분할
#X_train, X_test, y_train, y_test = train_test_split(concatenated_vectors, labels, test_size=0.2, random_state=42)

# 모델 초기화
#lr_p1 = LogisticRegression(C=0.1, penalty='l1', solver='saga', max_iter=1, warm_start=True)

# 손실 값 저장 리스트 초기화
#train_losses = []
#val_losses = []

# 반복 학습
#for i in range(10000):  # 10000번의 반복 학습
#    lr_p1.fit(X_train, y_train)
    
    # 훈련 손실 계산
#    y_train_pred_prob = lr_p1.predict_proba(X_train)
#    train_loss = log_loss(y_train, y_train_pred_prob)
#    train_losses.append(train_loss)
    
    # 검증 손실 계산
#    y_val_pred_prob = lr_p1.predict_proba(X_test)
#    val_loss = log_loss(y_test, y_val_pred_prob)
#    val_losses.append(val_loss)

# 손실 함수 시각화
#epochs = range(1, 10001)  # 100번의 반복 학습
#plt.plot(epochs, train_losses, 'bo-', label='loss')
#plt.plot(epochs, val_losses, 'ro-', label='val_loss')
#plt.title('loss')
#plt.xlabel('epoch')
#plt.ylabel('loss')
#plt.legend()
#plt.show()
